In [1]:
import pandas as pd
import numpy as np

In [2]:
ns = ['type', 'x', 'y', 'z']
# read points for joints
dfJointsVertexes = pd.read_csv('joints.obj', sep=' ', names = ns) 

In [6]:
# we will sort by 'x' axis as all points are located on the x axis
axis_sort = 'x' 
df = dfJointsVertexes.sort_values(by=[axis_sort])
# arms and elbows are first and last values after sort

dfJointsVertexesRowCount= len(dfJointsVertexes)
        
dfJointsVertexes = dfJointsVertexes.sort_values(by=[axis_sort])

idxWrist1 = 0
idxWrist2 = dfJointsVertexesRowCount - idxWrist1 - 1

idxElbow1 = 1
idxElbow2 = dfJointsVertexesRowCount - idxElbow1 - 1

idxShoulder1 = 2
idxShoulder2 = dfJointsVertexesRowCount - idxShoulder1 - 1

wrist1 = dfJointsVertexes.iloc[idxWrist1]
wrist2 = dfJointsVertexes.iloc[idxWrist2]

elbow1 = dfJointsVertexes.iloc[idxElbow1]
elbow2 = dfJointsVertexes.iloc[idxElbow2]

shoulder1 = dfJointsVertexes.iloc[idxShoulder1]
shoulder2 = dfJointsVertexes.iloc[idxShoulder2] 

In [9]:
# we have 2 vectors which show us the direction for the arms:
vector_l = np.asarray([wrist1.x-elbow1.x, wrist1.y-elbow1.y, wrist1.z-elbow1.z])
vector_r = np.asarray([wrist2.x-elbow2.x, wrist2.y-elbow2.y, wrist2.z-elbow2.z])

In [10]:
# we need to cut all the points which are lower than the joints for the elbow
# we take cosine similarities to measure which points to cut 
from sklearn.metrics.pairwise import cosine_similarity

# if the angle more than 90 degree, then we move the point to the place of an elbow joint


In [11]:
# read file with model 
dfModel = pd.read_csv('model.obj', sep = ' ', names = ['type', 'x', 'y', 'z'])

# vertexes
dfModelVertexes = dfModel[dfModel.type == 'v']
# points
dfModelFlats = dfModel[dfModel.type == 'f']

dfModelFlats['x'] = dfModelFlats.x.astype(int)
dfModelFlats['y'] = dfModelFlats.y.astype(int)
dfModelFlats['z'] = dfModelFlats.z.astype(int)

/home/yustiks/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/yustiks/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/home/yustiks/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

In [16]:
import math
dfModelVertexesRowCount = len(dfModelVertexes)

for idxVertex in range(0, dfModelVertexesRowCount):
    vertex = dfModelVertexes.iloc[idxVertex]    
    
    #left shoulder
    if vertex.x < shoulder1.x:
        elbow = elbow1
        normal = vector_l
    #right shoulder
    elif vertex.x > shoulder2.x:
        elbow = elbow2
        normal = vector_r
    else:
        continue
        
    vector = np.array([vertex.x - elbow.x, vertex.y - elbow.y, vertex.z - elbow.z])
        
    vectorLength = math.sqrt(vector[0]**2 + vector[1]**2 + vector[2]**2)
    
    scalarProduct = cosine_similarity(normal.reshape(1,-1), vector.reshape(1,-1))[0][0]
    
    if scalarProduct <= 0:
        continue
    
    # we take the projection of each point into the perpendicular vertex
    radius = math.sqrt(1 - scalarProduct**2) * vectorLength
    
    
    if radius > 0.1:
        continue
    
    dfModelVertexes.iloc[idxVertex] = elbow
        
dfModelVertexes.to_csv('model_new.obj', sep=' ', header=False, index=False, float_format='%.6f')
dfModelFlats.to_csv('model_new.obj', sep=' ', header=False, index=False, mode='a')
